## 1.1 一摞Python风格的纸牌
数据模型其实是对Python框架的描述，规范了这门语言自身构建模块的接口。\
利用特殊方法实现Python数据模型

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit']) # typename, 

beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [2]:
class FrenchDeck():
    ranks = [str(i) for i in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [3]:
deck = FrenchDeck()
print(len(deck))

52


In [4]:
deck[0]

Card(rank='2', suit='spades')

In [5]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [6]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

In [8]:
from random import choice
# 从序列中随机选一个元素

choice(deck)

Card(rank='3', suit='spades')

In [9]:
# 两种方式都可以
# for card in deck._cards:
#     print(card)

for card in deck:
    print(card)
    break

Card(rank='2', suit='spades')


迭代通常是隐式的，如果一个集合类型没有实现__contains__方法，那么in运算符就会按顺序做一次迭代搜索。

In [11]:
Card('A', 'hearts') in deck

True

排序

In [18]:
FrenchDeck.ranks

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [19]:
FrenchDeck.ranks.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [26]:
# 返回2的索引
print(FrenchDeck.ranks.index('2'))
print(FrenchDeck.ranks.index('A'))

0
12


In [27]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

# 返回card对应的排序优先度
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

for card in sorted(deck, key=spades_high):
    print(card)
    break

Card(rank='2', suit='clubs')


__len__和__getitem__的具体实现可以代理给self._cards这个python列表。

In [31]:
print(type(deck._cards))
print(type(deck))

print(deck[0])
print(deck._cards[0])

<class 'list'>
<class '__main__.FrenchDeck'>
Card(rank='2', suit='spades')
Card(rank='2', suit='spades')


In [33]:
# deck.sort() 不可行
deck._cards.sort() # 直接对列表进行操作可以，而且修改了deck的顺序

In [35]:
deck[0]

Card(rank='10', suit='clubs')

## 1.2 如何使用特殊方法
特殊方法是为了python解释器调用的，比如__len__，而我自己使用应该是len(my_object)。\
直接调用特殊方法的概率应该低于我自己实现它的概率，除了__init__。

### 1.2.1 模拟数值类型

In [38]:
print(abs(3))
print(abs(5.5))
# 如果是复数，返回复数的模

3
5.5


In [39]:
from math import hypot

In [58]:
# help(hypot) Return the Euclidean distance, sqrt(x*x + y*y).
class Vector():
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return('Vector(%d, %d)' % (self.x, self.y))
        
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, vector):
        return(Vector(self.x+vector.x, self.y+vector.y))
    
    def __mul__(self, scalar):
        return(Vector(self.x*scalar, self.y*scalar))

In [59]:
v1 = Vector(2,3)
v1 # python会调用__repr__

Vector(2, 3)

In [60]:
v2 = Vector(3,4)
v2

Vector(3, 4)

In [61]:
v1+v2

Vector(5, 7)

In [62]:
abs(v2)

5.0

In [63]:
v2*3

Vector(9, 12)

### 1.2.2 字符串表达形式
内置函数repr把一个对象以字符串形式表达出来以便辨认，即“字符串表达形式”，如果没有实现__repr__，可能打印的结果是

%格式化字符串是老的用法，str.format是新的用法，都利用了repr函数来获取对象的标准字符串表示形式。目前是两种用法并存的局面。\
__repr__返回的字符串应该准确无歧义，尽可能表达该被打印的对象是如何创建的，本例采用类似调用对象构造器的表达形式。\
__repr__和__str__函数的区别是，后者在str函数使用的时候被调用，输出更为友好。但如果只实现一个，建议实现__repr__，因为如果没有__str__，而python又需要使用时会用__repr__代替。

### 1.2.3 算术运算符
\+ \* 等中缀运算符的基本原则是不改变操作对象，而是产出一个新值。

### 1.2.4 自定义的布尔值
为了判断一个对象是真还是假，python会调用bool(x)。一般情况下，我们自定义的类的实例被认为是真的，除非自己定义了__bool__或者__len__函数，如果没有__bool__，python会调用__len__，如果返回0，则bool会返回false。

## 1.3 特殊方法一览
当交换两个操作数的位置时，会调用反向算术运算符，比如__radd__。\
增量赋值运算符是一种把中缀运算符变成赋值运算的捷径(a = a*b变成a *= b)。

## 1.4 为什么len不是普通方法？
实用剩于纯粹。如果x是一个内置类型的实例，CPython会直接从一个C结构体中读取对象的长度，完全不会调用任何方法，这是很快的，而这一操作很常见。\
又由于len是特殊方法，我们也可以用于自定义数据类型，因此在保持内置类型的效率和保证语言一致性间找到了一个平衡点。

## 1.5 本章小结
通过实现特殊方法，自定义数据类型可以表现得跟内置类型一样，从而写出更加pythonic的代码。\
python对象的一个基本要求是有合理的字符串表示形式，可以用__repr__和__str__来实现。\
对序列数据类型的模拟是特殊方法使用最多的地方。\
python通过运算符重载提供了丰富的数据类型，比如本章实现的Vector类。